# Training Deep Neural Networks (DNNs)
To tackle complex problems we often need deeper models with 10+ hidde layers, hundreds of neurons per layer, linked by hundreds of thousands of connections.This can cause some common challenges:
- Vanishing Gradients Problem -> many solutions below
- Exploding Gradients Problem -> many solutions below
- Lack of Training Data and Overfitting Noisy Data -> transfer learning and unsupervised pretraining
- Slow Training -> choice of optimizer

# 1. Vanishing and Exploding Gradients
The backpropagation algorithm works by going from the output layer to the input layer and computing the gradient of the cost function with regard to each parameter of the network, then using these gradients to update the parameter with Gradient Descent.

As the algorithm progresses backwards, gradients often get smaller towards the earlier layers and Gradient Descent barely updates these weights. The reason is that chain rule is used to calculate the gradients, multiplying a bunch of small values <1 within the formula, and thus make the overall results small as well.

Since the weight update is proportional to the gradient size, the update of early layer weights is very small, which hinders learning and training never converges to a good solution. This is called the **vanishing gradients problem**. 

In some cases, the opposite can happen and gradients become larger and larger, leading to overadjust the weights in the earlier layers of the network. We would move further and further away from the optimal weights and the algorithm diverges. This problem is called **exploding gradients** and is quite common for recurrent neural networks (RNNs).

Researchers in 2010 found the combination of logistic sigmoid activation function and a normal-distribution weight initialization technique to be a common reason for this behavior. The result is that the variance of each layer's output is larger than for its input.

#### Choice of Initialization
Glorot and He propose an initialization techniques where input and output variance stays equal when moving forward (like a chain of microphones and amplifiers), and the variance of the gradients to stay equal before and after moving through the layers in backpropagation. Both cannot be guaranteed but the **Xavier or Glorot Initialization** found a good compromise that works well in practice. It is very similar to LeCun Initialization (1990s).

Keras uses Glorot initialization with a uniform distribution by default. This behavior can be changed using kernal_initialization.

#### Nonsaturating Activation Functions
Next to initialization the choice of activation functions can lead to saturations. ReLu is great in general since it does not saturate for positive values and is fast to compute. However, some neurons can die turing training, meaning they stop outputting anything other than 0. This problem is known as **dying ReLU**. 

The variant called **LeakyReLU** avoids this problem by having a non-zero slope even for negative values. There are many types of such leaky ReLus and we can influence the leakiness by tweaking the hyperparameter alpha. 

Alternatively, we can use the **ELU function** which looks like a smoothed ReLu. Usually, ELU leads to longer training time than ReLu. There is a variant called SELU which can lead to the network's self-normalization given certain prerequesits.

Geron suggests: SELU > ELU > Leaky ReLU > ReLU > tanh > logistic

In [ ]:
# LeakyReLU
model = keras.models.Sequential([
    [...],
    keras.layers.Dense(30, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2)
    [...]
])

In [ ]:
# SELU
model = keras.models.Sequential([
    [...],
    keras.layers.Dense(30, activation="selu", kernel_initializer="lecun_normal"),
    [...]
])

### Batch Normalization
Proper initialization prevents the vanishing/exploding gradients problem in the beginning of training the network. However, it can occur later. **Batch Normalization (BN)** addresses this problem. 

An operation is added just before and after the activation function of each hidden layer that zero-centers and normalizes each input, then scales and shifts the result. Thus, the operation leds the model learn the optimal scale and mean of each of the layer's input.

Adding a BN layer as the very first layer is roughly equivalent to standardizing the data before training. 

Four parameter vectors are learning in keras' standard implementation of Batch Normalization: 
- the output scale vector (backpropagation)
- the output offset vector (backpropagation)
- the final input mean vector (estimated during training but used only after training)
- the final input standard deviation vector (estimated during training but used only after training)

Researchers generally found **Batch Normalization to improve all kinds of DNNs**,, e.g. lead to huge improvements on the ImageNet classification task. Using more saturating activation functions like tanh and logistic activation function become possible again. Furthermore, BN acts as a regulizer, reducing the need for dropouts.

Downsides are slower predictions and slightly more complex models. Epochs are generally found to be slower, however, fewer epochs are usually needed for convergence. Thus, wall time will usually be shorter.

Applying BN after each layer is so common in practice that the BN layers are hidden in diagrams. New bleeding-edge research challenges this approach but as of today it is still good practice.

In [ ]:
# BN as first layer and after every hidden layer
# usually necessary for way deeper networks
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28])
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation_function="softmax")
])

### Gradient Clipping
Another approach to avoid exploding gradients is to clip gradients during backpropagation by using a threshold. This approach is an alternative to BN in RNNs, for other types BN is sufficient.


In [1]:
optimizer = keras.optimizers.SGD(clipvalue=1.0) # orientation of the gradient might change; instead clipnorm=1.0
model.compile(loss="mse", optimizer=optimizer)

NameError: name 'keras' is not defined

# 2. Reusing Pretrained Layers

Chapter 14 shows ways of finding a good existing neural network. We shouldn't train a deep network from scratch but instead reuse the lower layers of an existing network. This technique is called **transfer learning**.

For example: We want to use Model A, which was used for a similar task, and extend it for our current task. We load the model, remove the last layer, and replace it by a new one that suits the problem (e.g., binary classification).

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

Training the new model will change the weights of the old model. We want to avoid that and use the results from pretraining. Therefore we need to clone model A first and copy the weights.

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

Training the new model on the new task can lead to large error gradients that could destroy the reused weights. Therefore it is adviced to freeze the reused layers during the first few epochs. 

Note: Freezing and unfreezing requires to compile the model.

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy", optimize="sgd",
                    metrics=["accuracy"])

We can then train the new model for a few epochs, change trainability of layers back to true, and continue. A good approach is to reduce the learning rate after unfreezing.

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                          validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = keras.optimizers.SGD(lr=0.0001)

model_B_on_A.compile(loss="binary_crossentropy", optimize="sgd",
                    metrics=["accuracy"])

history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                          validation_data=(X_valid_B, y_valid_B))

Note: Transfer learning does not work with small dense networks but best with deep CNNs (Chapter 14).

# 3. Faster Optimizers

To summarize, speeding up training can be achieved by the using:
- good initialization strategy for the connection weights
- good activation function
- Batch Normalization
- reusing parts from pre-trained models

Additionally, we can speed training up by using faster optimizers than regular SGD.
- momentum optimization
- Nesterov Accelerated Gradient (NAG)
- AdaGrad
- RMSProp
- Adam and Nadam (adam+nag) optimization

Advantages and Disadvantages on p.359

In [ ]:
# momentum optimizer
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

# NAG
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

# RMSprop
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9) # replaced by Adam optimizers

# Adam
optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

# 4. Learning Rate Scheduling

A common starting approach when searching for a good learning rate is to start very small, train for a few hundred iterations, exponentially increase lr, repeat to very large lr values, then plot the loss as a function of the learning rate, and choose the lr just before the loss curve shoots up again.

This approach uses a constant learning rate. We could reach a good solution faster by using a dynamic learning rate, for exampple bigger in the beginning and smaller later. There are many such strategies calles **learning rate scheduling**.

- power scheduling
- exponential scheduling
- piecewise constant scheduling
- performance scheduling
- icycle scheduling

# 5. Avoiding Overfitting through Regularization
- L1 and L2 regularization
- Dropout
- Monte Carlo (MC) Dropout
- Max-Norm regularization

# 6. Practical Guidelines

Default DNN configuration:

    Kernel initializer: He initialization
    Activation function: ELU
    Normalization: None if shallow, Batch Norm if deep
    Regularization: Early stopping +L2 if needed
    Optimizer: Momentum optimization like Nadam or RMSProp
    Learing rate scheduler: 1cycle

Don't forget to normalize the input features and to use parts of pre-trained models if possible.